In [ ]:
import json
import pandas as pd
import time
# data = [json.loads(line) for line in open(r'C:\Users\Mhaiskao\Desktop\Assignments\MSc Project\tiktok.json', 'r',encoding="utf8")]
data = [json.loads(line) for line in open(r'C:\Users\Mhaiskao\Desktop\Assignments\MSc Project\scraper\tiktok.json', 'r',encoding="utf8")]

In [ ]:
print(data[84594])

In [ ]:
#################################IMp###################

In [ ]:
from dateutil import parser
id_list=[]
text_list=[]
Music_id_list=[]
Music_name_list=[]
Author_id_list=[]
Created_time=[]
verified_list=[]

sharecount_list=[]
digcount_list=[]
commentcount_list=[]
scrape_time_list=[]
weekday_list=[]

playcount_list=[]
final_comment_count=[]
final_share_count=[]
final_digg_count=[]
final_play_count=[]
bin_time_list=[]
challenge_name_list=[]
challenge_id_list=[]

for i in range(0,len(data)):
    id_list.append(data[i]['tiktokData'][0]['itemInfos']['id'])
    text_list.append(data[i]['tiktokData'][0]['itemInfos']['text'].lower())
    Music_id_list.append(data[i]['tiktokData'][0]['musicInfos']['musicId'])
    Music_name_list.append(data[i]['tiktokData'][0]['musicInfos']['musicName'])
    Author_id_list.append(data[i]['tiktokData'][0]['itemInfos']['authorId'])
    created_date=parser.isoparse(data[i]['createdAt']['$date'])
    Created_time.append(created_date.strftime("%x"))
    verified_list.append(data[i]['tiktokData'][0]['authorInfos']['verified'])
    
    #Save challenges Name and Challenge Id for every video
    
    challenge_name=[]
    challenge_id=[]
    
    #Check if no challenges are present
    
    if len(data[i]['tiktokData'][0]['challengeInfoList']) == 0:
        challenge_id.append('Empty')
        challenge_name.append('Empty')
    else:
        for j in range(0,len(data[i]['tiktokData'][0]['challengeInfoList'])):
            challenge_id.append(data[i]['tiktokData'][0]['challengeInfoList'][j]['challengeId'])
            challenge_name.append(data[i]['tiktokData'][0]['challengeInfoList'][j]['challengeName'])
    challenge_id_list.append(challenge_id)
    challenge_name_list.append(challenge_name)
    
    #Get Share, Comment and Digg Count 
    
    sharecount=[]
    digcount=[]
    commentcount=[]
    scrape_time=[]
    playcount=[]
    bin_time=[]
    weekday=[]
    for j in range(0,len(data[i]['tiktokData'])):   
        date=parser.isoparse(data[i]['tiktokData'][j]['scrapedAt']['$date'])
        digcount.append(data[i]['tiktokData'][j]['itemInfos']['diggCount'])
        sharecount.append(data[i]['tiktokData'][j]['itemInfos']['shareCount'])
        commentcount.append(data[i]['tiktokData'][j]['itemInfos']['commentCount'])
        scrape_time.append(date.strftime("%x %H"))
        if date.hour in range(0,6):
            bin_time.append("{} , {}".format(date.strftime("%x %w"),'Part 1'))
            weekday.append(date.strftime("%w"))
#             part.append('Part 1')
        elif date.hour in range(7,12):
            bin_time.append("{} , {}".format(date.strftime("%x %w"),'Part 2'))
            weekday.append(date.strftime("%w"))
#             part.append('Part 2')
        elif date.hour in range(13,18):
            bin_time.append("{} , {}".format(date.strftime("%x %w"),'Part 3'))
            weekday.append(date.strftime("%w"))
#             part.append('Part 3')
        else:
            bin_time.append("{} , {}".format(date.strftime("%x %w"),'Part 4'))
            weekday.append(date.strftime("%w"))
#             part.append('Part 4')
        try:
            playcount.append(data[i]['tiktokData'][j]['itemInfos']['playCount'])
        except KeyError:
            playcount.append(0)
    weekday_list.append(weekday)
    bin_time_list.append(bin_time)        
    sharecount_list.append(sharecount)
    digcount_list.append(digcount)
    commentcount_list.append(commentcount)
    final_comment_count.append(commentcount[-1])
    scrape_time_list.append(scrape_time)
#     part_list.append(part)
    weekday_list.append(weekday)
    playcount_list.append(playcount) 
    final_share_count.append(sharecount[-1])
    final_digg_count.append(digcount[-1])
    final_play_count.append(playcount[-1])
    
df_final=pd.DataFrame(list(zip(id_list,text_list,Music_id_list,Music_name_list,Author_id_list,Created_time,verified_list,challenge_id_list,
                               challenge_name_list,sharecount_list,final_share_count,digcount_list,final_digg_count,commentcount_list,
                               final_comment_count,scrape_time_list,weekday_list,bin_time_list,playcount_list,final_play_count)),
                       columns=['ID','Text','Music Id','Music Name','Author ID','Created Time','Verified List','challenge_list',
                                'challenge_list_Name','Sharecount List','Final Share Count','Diggcount List','Final Digg Count',
                                'Commentcount List','Final Comment Count','Scrape_time List','weekday_list','Bin Time','Playcount List','Final Play Count'])



In [ ]:
df_final


Get author id for specific commencial

In [ ]:
from statistics import mean
def hashtag(sub):
# creating and passsing series to new column 
    df_sub_string=df_final[df_final["Text"].str.contains(sub,regex=True)]
    df_sub_string.reset_index(inplace=True,drop=True)

    return(df_sub_string)

Time Series

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import itertools
import warnings
import numpy as np
warnings.filterwarnings('ignore')

arima_df=pd.DataFrame()
mean_comment=pd.DataFrame()
mean_comment['Time']=arima_df['time']=df_final['Bin Time'][0]
mean_comment['Weekday']=df_final['weekday_list'][0]
mean_comment['likes']=df_final['Diggcount List'][0]
mean_comment['Per Change']=mean_comment['likes'].diff()
mean_comment=mean_comment.groupby(['Time'])['Per Change'].sum()

#Convert to DataFrame
frame = { 'Time Data': mean_comment.index, 'Likes Value': mean_comment.values } 
result = pd.DataFrame(frame) 
weekday=[]
for i in range(0,len(result)):
    string=result['Time Data'][i]
    string1=string.split()
    weekday.append(int(string1[1]))
result['Weekday']=weekday

TrainingSize = int(0.8*result.shape[0])
TestSize = result.shape[0] - TrainingSize
exog_value=result.iloc[0:TrainingSize,2]

p=q=d=range(0,1)
pdq=list(itertools.product(p,d,q))
seasonal_pdq = [(x[0], x[1], x[2], 4) for x in list(itertools.product(p, d, q))]
mean_squared=[]
pdq_values=[]
aic=[]

for param in pdq:
    for param2 in seasonal_pdq:
        try:
            #Define seasonal and normal parameter
            my_order = param
            my_seasonal_order = param2

            #Train the model
            model = SARIMAX(result.iloc[0:TrainingSize,1], 
                            order=my_order, seasonal_order=my_seasonal_order,
                            exog=exog_value,simple_differencing=True)
#             model = SARIMAX(result.iloc[0:TrainingSize,1], 
#                             order=my_order, seasonal_order=my_seasonal_order,simple_differencing=True)
            #Fit the model
            model_fit = model.fit()        

            #Forecast for next Test Size
            yhat =model_fit.forecast(TestSize,exog=pd.DataFrame(result['Weekday'])[TrainingSize:])

            #Calculate mean squared error
            mean_squared.append(mean_squared_error(np.ceil(yhat),result.iloc[TrainingSize:,1]))

            pdq_values.append((param,param2))
            aic.append(model_fit.aic)
        except:
            print('In except')
            continue
            
mydict=dict(zip(pdq_values,mean_squared))
mydict2=dict(zip(pdq_values,aic))

Keymax = min(mydict, key= lambda x: mydict[x])
print(Keymax)
my_order,my_seasonal_order=Keymax

#Train model
model = SARIMAX(result.iloc[0:TrainingSize,1], 
                            order=my_order, seasonal_order=my_seasonal_order,
                            exog=exog_value)
# model = SARIMAX(result.iloc[0:TrainingSize,1], 
#                             order=my_order, seasonal_order=my_seasonal_order)

model_fit = model.fit()
yhat =model_fit.forecast(TestSize,exog=pd.DataFrame(result['Weekday'])[TrainingSize:])

print(mean_squared_error(np.ceil(yhat),result.iloc[TrainingSize:,1]))
print(yhat)
print(result.iloc[TrainingSize:,1])

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

value=mean_absolute_percentage_error(result.iloc[TrainingSize:,1],yhat)
# mean_comment.plot.bar(figsize=(30,10))
print(value)

In [ ]:
result

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta
import itertools
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def visualize_arima(output,time,mean_comment):
    #save the last date and use stripftime to generate 7 days from last date
    date_time_str = time[-1]
    Predicted_Value_dates=[]

    #check if length of input was 1
    if len(mean_comment)==1:
        for i in range(1,8):
            res = (datetime.strptime(date_time_str, '%m/%d/%y %H') + timedelta(days=i)).strftime('%m/%d/%y %H')
            Predicted_Value_dates.append((res,output))   
    else:
        for i in range(1,8):
            res = (datetime.strptime(date_time_str, '%m/%d/%y %H') + timedelta(days=i)).strftime('%m/%d/%y %H')
            Predicted_Value_dates.append((res,output[i-1]))

    #create data frame to plot the graphs
    df1=pd.DataFrame()
    df=pd.DataFrame()
    df[['Date','Predicted_Value']]=pd.DataFrame(Predicted_Value_dates)

    mean_comment_time=[]
    for i in range(len(mean_comment)):
        mean_comment_time.append((mean_comment.index[i],mean_comment[i]))

    df1[['Date','Orignal_Value']]=pd.DataFrame(mean_comment_time)

    # Initialize figure with subplots
    fig = make_subplots(rows=1, cols=2)


    # Add Comment Count Scatter chart
    fig.add_trace(go.Scatter(x=df1['Date'], y=df1['Orignal_Value'], name='Orignal Values',
                             line = dict(color='red', width=2)),row=1,col=1)

    # Add Share Count Scatter chart
    fig.add_trace(go.Scatter(x=df['Date'], y=df['Predicted_Value'], name='Predicted Values',
                             line = dict(color='blue', width=2)),row=1,col=2)


    return fig


#######################################################################################################################

def arima_model(sub_string,index):
    parameters=[]
    aic=[]
    mydict={}
    
    arima_df=pd.DataFrame()
    Time=arima_df['time']=sub_string['Scrape_time List'][index]
    arima_df['comment']=sub_string['Commentcount List'][index]
    
    mean_comment=arima_df.groupby('time')['comment'].mean()
    mean_comment=np.floor(mean_comment)

    p=q=d=range(0,5)
    pdq=list(itertools.product(p,d,q))
    
    for param in pdq:
        # fit model
        try:
            model = ARIMA(mean_comment, order=param)
            model_fit = model.fit(disp=0)
            output=model_fit.forecast(steps=2)[0]
            parameters.append(param)
            aic.append(model_fit.aic)
            
        except:
            continue
    
    mydict=dict(zip(parameters,aic))
    if len(mydict)==0:

        return visualize_arima(mean_comment[0], Time, mean_commen)
    
    Keymax = min(mydict, key= lambda x: mydict[x])
    model = ARIMA(mean_comment, order=Keymax)
    model_fit = model.fit(disp=0)
    output=model_fit.forecast(steps=7)[0]
    
    return visualize_arima(output,Time,mean_comment)
##############################################################
sub_string=hashtag('lol')
fig=arima_model(sub_string,0)
fig.show()

import plotly.io as pio
pio.write_html(fig, file='arima_visualization.html', auto_open=False)


Potential Influencers:  value= [(sharecount + diggcount + commentcount)/playcount] * 100

In [ ]:
def Calculate_Engagement_rate(df_sub_string,authorid):
    rslt_df = df_sub_string.loc[df_sub_string['Author ID'] == authorid]
    rslt_df.reset_index(inplace=True,drop=True)
    rslt_df.sort_values("Created Time", axis = 0, ascending = False, inplace = True) 
    
    #Calculate enagament rate using formular ER=[(sharecount + diggcount + commentcount)/playcount] * 100
    Numerator=rslt_df['Final Share Count'] + rslt_df['Final Comment Count'] + rslt_df['Final Digg Count']
    Numerator_value=sum(np.array(Numerator))
    Denominator=rslt_df['Final Play Count']
    Denominator_value=sum(np.array(Denominator))
    Engagement_rate= (Numerator_value/Denominator_value)*100
    Engagement_rate
    return Engagement_rate

def potential_influencers(hashtag_string):
    Engagement_rate_list=[]
    Engagement_rate_list_complete=[]
    Author_ID_list=[]
    
    df_sub_string=hashtag(hashtag_string.lower())
    
    #Influencers with maximum videos
    df_top_influencers=pd.DataFrame()
    df_top_influencers['Author ID']=df_sub_string.groupby("Author ID",sort=False)["ID"].count().sort_values(ascending=False)
    
    for i in range(0,len(df_top_influencers)):
        #save Engagement rate for each author
        Engagement_rate_list.append(Calculate_Engagement_rate(df_sub_string,df_top_influencers['Author ID'].index[i]))
        Engagement_rate_list_complete.append(Calculate_Engagement_rate(df_final,df_top_influencers['Author ID'].index[i]))
        #save author id
        Author_ID_list.append(df_top_influencers['Author ID'].index[i])
    return Engagement_rate_list,Author_ID_list,Engagement_rate_list_complete

ER,Author_ID,ER_complete=potential_influencers('puma')
df_influencers=pd.DataFrame()
df_influencers[['ER','Author_ID']]=pd.DataFrame(list(zip(ER,Author_ID)))
df_influencers.sort_values('ER', inplace=True, ascending=False)

################
df_influencers_complete=pd.DataFrame()
df_influencers_complete[['ER','Author_ID']]=pd.DataFrame(list(zip(ER_complete,Author_ID)))
df_influencers_complete.sort_values('ER', inplace=True, ascending=False)
df_influencers_complete

In [ ]:
df_influencers

In [ ]:
df_final['Text'].loc[df_final['Author ID'] == '6613398665766305798']

In [ ]:
data[84594]

In [ ]:
df_text=df_final[['Music Id','Author ID','challenge_list_Name','Verified List']]

# df_text=df_text.rename(columns = {'Author ID':'source'})
# df_text=df_text.rename(columns = {'challenge_list_Name':'target'})


In [ ]:
def remove_zero_challenges(df_final):
    #Remove rows having Zero challenges 
    df_text=df_final[['Music Id','Author ID','challenge_list_Name','Verified List']]
    drop_index=[]
    for i in range(0,len(df_text)):
        if df_text['challenge_list_Name'][i]==['Empty']:
            drop_index.append(i)
    df_text.drop(df_text.index[drop_index],inplace=True)
    df_text.reset_index(drop=True, inplace=True)
    return df_text

# df_text=remove_zero_challenges(df_text)
# df_text.head()

In [ ]:
def challenges_for_verified_users(df_text):
    #Challenges associated with verifed users
    listauthor=[]
    author=[]
    listauthor=df_text.index[df_text['Verified List'] == True].tolist()
    for i in listauthor:
        author.append([df_text['Author ID'][i],df_text['Music Id'][i],df_text['challenge_list_Name'][i]])
    df_verified_user_challenges=pd.DataFrame()
    df_verified_user_challenges[['Author ID','Music Id','challenge_list_Name']]=pd.DataFrame(author)
    return df_verified_user_challenges

df_verified_user_challenges=challenges_for_verified_users(remove_zero_challenges(df_final))
df_verified_user_challenges.head()

In [ ]:
#Create list having format author : challenge1 for force directed graph  

def create_df_graph(df_text):
    append_list=[]
    for i in range(0,len(df_text['Author ID'])): 
        for j in df_text['challenge_list_Name'][i]:       
             append_list.append([df_text['Music Id'][i],df_text['Author ID'][i],j,1])
    cols = ['Music ID','Author ID','challenge_list_Name','weight']

    df_graph = pd.DataFrame(append_list,columns=cols)  
    return df_graph

def unique_values(dataframe,columnname):
    dataframereturn=pd.DataFrame()
    dataframereturn=dataframe[columnname].value_counts().rename_axis('unique_values').reset_index(name='counts')
    return dataframereturn

#Unique values for Author ID
df_bar_author_id=unique_values(df_final,'Author ID')

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def visualize_CMD(df_final,tiktokid):
    index=df_final.index[df_final['ID'] == tiktokid].tolist()
    comment_count=df_final['Commentcount List'][index[0]]
    share_count=df_final['Sharecount List'][index[0]]
    play_count=df_final['Playcount List'][index[0]]
    digg_count=df_final['Diggcount List'][index[0]]
    scrape_time=df_final['Scrape_time List'][index[0]]

    # Initialize figure with subplots
    fig = make_subplots(
        rows=3, cols=2,
        column_widths=[0.9, 0.9],
        row_heights=[0.5, 0.2,0.5],
        specs=[[{"type": "Scatter"}, {"type": "Scatter"}],
               [None , None ],
               [{"type": "Scatter"}, {"type": "Scatter"}]])


    # Add Comment Count Scatter chart
    fig.add_trace(go.Scatter(x=scrape_time, y=comment_count, name='Comment Count',
                             line = dict(color='red', width=2)),row=1,col=1)

    # Add Share Count Scatter chart
    fig.add_trace(go.Scatter(x=scrape_time, y=share_count, name='Share Count',
                             line = dict(color='blue', width=2)),row=1,col=2)

    # Add Digg Count Scatter chart
    fig.add_trace(go.Scatter(x=scrape_time, y=digg_count, name='Digg Count',
                             line = dict(color='green', width=2)),row=3,col=1)

    # Add Playcount Count Scatter chart
    fig.add_trace(go.Scatter(x=scrape_time, y=play_count, name='Play Count',
                             line = dict(color='black', width=2)),row=3,col=2)

    return fig

fig=visualize_CMD(df_final,'6789793077612547333')
fig.show()

In [ ]:
#Plot bar grpah for top 20 CHALLENGES

import plotly.express as px
def visualize_top20_challenge(df_graph):
    df_bar_challenge=unique_values(df_graph,'challenge_list_Name')
    fig = px.bar(df_bar_challenge[0:20], y='counts', x='unique_values', text='counts')
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    return fig
fig=visualize_top20_challenge(create_df_graph(remove_zero_challenges(df_final)))
fig.show()

In [ ]:
#Plot bar graph for top 20 Music NAMES
import plotly.express as px
def visualize_top20_MusicName(df_final):
    #Unique values for Music Names
    df_bar_music_name=unique_values(df_final,'Music Name')
    #Skipped 1st music name since random orignal music is represented as 'orginal music'. 
    # Thus the category 'orginal music' represent multiple songs
    fig = px.bar(df_bar_music_name[1:20], y='counts', x='unique_values',text='counts')
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    return fig

fig=visualize_top20_MusicName(df_final)
fig.show()

In [ ]:
#Bar graph for top trensding music id using plotly

import plotly.graph_objects as go
import plotly.figure_factory as ff
def visualize_top20_MusicID(df_final):
    # #Unique values for Music Ids
    df_bar_music_id=unique_values(df_final,'Music Id')
    index=list(range(10))
    music_id = list(df_bar_music_id['unique_values'][0:10])
    counts = list(df_bar_music_id['counts'][0:10])
    Table=[]
    Table.append(['Index','Music Id','Count'])
    for  i in range(0,len(music_id)):
        Table.append([i,music_id[i],counts[i]])

    fig = ff.create_table(Table, height_constant=60)

    Music_Bar = go.Bar(x=index, y=counts, xaxis='x2', yaxis='y2',
                    marker=dict(color='#0099ff'),
                    name='Count')

    fig.add_traces(Music_Bar)

    # initialize xaxis2 and yaxis2
    fig['layout']['xaxis2'] = {}
    fig['layout']['yaxis2'] = {}

    # Edit layout for subplots
    fig.layout.xaxis.update({'domain': [0, .5]})
    fig.layout.xaxis2.update({'domain': [0.6, 1.]})

    # The graph's yaxis MUST BE anchored to the graph's xaxis
    fig.layout.yaxis2.update({'anchor': 'x2'})
    fig.layout.yaxis2.update({'title': 'Count'})
    fig.layout.xaxis2.update({'title': 'index'})

    fig.layout.margin.update({'t':75, 'l':50})
    fig.layout.update({'title': 'Trending Music on TikTok'})
    return fig

fig=visualize_top20_MusicID(df_final)
fig.show()

In [ ]:
#Bar graph for top 10 challenge combinations

import plotly.graph_objects as go
import plotly.figure_factory as ff
def visualize_top10_challenge_combination(df_final):
    #Unique values for Challenge combination
    df_bar_combination=unique_values(remove_zero_challenges(df_final),'challenge_list_Name')
    df_bar_combination1=df_bar_combination[df_bar_combination['unique_values'].apply(lambda x: len(x)>1)].reset_index(drop=True)
    index=list(range(10))
    challenge_id = list(df_bar_combination1['unique_values'][0:10])
    counts = list(df_bar_combination1['counts'][0:10])
    Table=[]
    Table.append(['Index','Challenges','Count'])
    for  i in range(0,len(challenge_id)):
        Table.append([i,challenge_id[i],counts[i]])

    fig = ff.create_table(Table, height_constant=60)
    fig.layout.width=1000
    Challenge_Combination_Bar = go.Bar(x=index, y=counts, xaxis='x2', yaxis='y2',
                    marker=dict(color='#0099ff'),
                    name='Count')

    fig.add_traces(Challenge_Combination_Bar)

    # initialize xaxis2 and yaxis2
    fig['layout']['xaxis2'] = {}
    fig['layout']['yaxis2'] = {}

    # Edit layout for subplots
    fig.layout.xaxis.update({'domain': [0, .5]})
    fig.layout.xaxis2.update({'domain': [0.6, 1.]})

    # The graph's yaxis MUST BE anchored to the graph's xaxis
    fig.layout.yaxis2.update({'anchor': 'x2'})
    fig.layout.yaxis2.update({'title': 'Count'})
    fig.layout.xaxis2.update({'title': 'index'})

    fig.layout.margin.update({'t':75, 'l':50})
    fig.layout.update({'title': 'Trending Challenge Combination on TikTok'})
    return fig

fig=visualize_top10_challenge_combination(df_final)
fig.show()

In [ ]:
#Connected graph displaying Challenges associated with Music across dataset

import d3fdgraph
import pandas
def visualize_hashtag_for_music(df_graph,musicid):
    listauthor=df_graph.index[df_graph['Music ID'] == musicid].tolist()
    cols = ['source','target','weight']
    list_directed_graph=[]
    #Create list having format Music Id : challenge1;  
    for i in listauthor:
        list_directed_graph.append([df_graph['Music ID'][i],df_graph['challenge_list_Name'][i],5])

    df_directed_graph=pd.DataFrame(list_directed_graph,columns=cols)
    d3fdgraph.plot_force_directed_graph(df_directed_graph, node_radius=15, link_distance=30, collision_scale=4)
    
visualize_hashtag_for_music(create_df_graph(remove_zero_challenges(df_final)),'6735137560026172166')


############## An alternate to website : Interactive plots (just an example)#########

In [ ]:
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets
df = pd.read_csv('https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv')
df = df.drop(df.columns[[0]], axis=1)
df['carrier'].unique()


In [ ]:
month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

container = widgets.HBox(children=[use_date, month])

textbox = widgets.Dropdown(
    description='Airline:   ',
    value='DL',
    options=df['carrier'].unique().tolist()
)

origin = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)


# Assign an empty figure widget with two traces
trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='NYC FlightDatabase'
                        ),
                        barmode='overlay'
                    ))

In [ ]:
def validate():
    if origin.value in df['origin'].unique() and textbox.value in df['carrier'].unique():
        return True
    else:
        return False


def response(change):
    if validate():
        if use_date.value:
            filter_list = [i and j and k for i, j, k in
                           zip(df['month'] == month.value, df['carrier'] == textbox.value,
                               df['origin'] == origin.value)]
            temp_df = df[filter_list]

        else:
            filter_list = [i and j for i, j in
                           zip(df['carrier'] == 'DL', df['origin'] == origin.value)]
            temp_df = df[filter_list]
        x1 = temp_df['arr_delay']
        x2 = temp_df['dep_delay']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Delay in Minutes'
            g.layout.yaxis.title = 'Number of Delays'


origin.observe(response, names="value")
textbox.observe(response, names="value")
month.observe(response, names="value")
use_date.observe(response, names="value")

In [ ]:
container2 = widgets.HBox([origin, textbox])
widgets.VBox([container,
              container2,
              g])